# Market-Making & ETF-Hedging Simulator (Public Data)

**Content**
- Part 1 — Quoting Engine on Streaming Prices  
- Part 2 — Risk-Aware Skewing & Position Limits  
- Part 3 — ETF Hedging via Rolling OLS Betas  
- Part 4 — (Optional) Simple Stat-Arb Spread Signal  
- Evaluation: out-of-sample split, costs/slippage, Sharpe, Sortino, max drawdown, turnover

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List, Tuple, Dict, Iterable, Optional
import yfinance as yf 
import random
    
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)

In [ ]:
def get_price_series(tickers: List[str], interval: str = "1d", period: str = "2y") -> pd.DataFrame:
    data = yf.download(tickers, interval=interval, period=period, auto_adjust=True, progress=False)
    if isinstance(data.columns, pd.MultiIndex):
        data = data['Close']
    data = data.dropna(how="all").dropna(axis=1, how="any")
    return data

def build_price_requests(prices: pd.DataFrame, lot: int = 100) -> List[Tuple[str, int, float, pd.Timestamp]]:
    out = []
    last_rows = min(500, len(prices))
    for t in prices.columns:
        series = prices[t].iloc[-last_rows:].dropna()
        for ts, px in series.items():
            out.append((t, lot, float(px), pd.Timestamp(ts)))
    return out

In [ ]:
from dataclasses import dataclass
import pandas as pd
import random

@dataclass
class CompletedTrade:
    ticker: str
    side: str          # "BUY" or "SELL"
    bid_price: float
    offer_price: float
    ref_price: float
    date: pd.Timestamp
    volume: int


class SimpleMarketMaker:
    def __init__(self, base_spread_bps=10.0, pos_limit=2000, skew_bps_at_limit=50.0):
        self.base_spread_bps = base_spread_bps
        self.pos_limit = pos_limit
        self.skew_bps_at_limit = skew_bps_at_limit
        self.position = {}
        self.cash = 0.0
        self.completed_trades = []
        self.current_positions = {}
        self.quoted_trades = []
        self.hedge_trades = []

    def add_trade(self, trade):
        self.completed_trades.append(trade)

    def add_quoted_trade(self, quote):
        self.quoted_trades.append(quote)

    def quote(self, ticker, ref_price):
        pos = self.position.get(ticker, 0)
        skew_frac = pos / max(1, self.pos_limit)
        skew_bps = skew_frac * self.skew_bps_at_limit

        half_spread = (self.base_spread_bps / 2) / 1e4 * ref_price
        skew = (skew_bps / 1e4) * ref_price

        bid = ref_price - half_spread - skew
        offer = ref_price + half_spread - skew
        return bid, offer

    def fill_against_stream(self, ticker, ref_px, vol, ts):
        bid, offer = self.quote(ticker, ref_px)

        # Randomly simulate taker fills
        if random.random() < 0.5:  # 50% chance of trade
            if random.random() < 0.5:
                side = "BUY"
                self.position[ticker] = self.position.get(ticker, 0) + vol
                self.cash -= bid * vol
            else:
                side = "SELL"
                self.position[ticker] = self.position.get(ticker, 0) - vol
                self.cash += offer * vol

            # Save trade with all required fields
            self.completed_trades.append(
                CompletedTrade(
                    ticker=ticker,
                    side=side,
                    bid_price=bid,
                    offer_price=offer,
                    ref_price=ref_px,
                    date=pd.Timestamp(ts),
                    volume=vol
                )
            )

    def mark_to_market(self, marks: dict):
        eq = self.cash
        for t, q in self.position.items():
            if t in marks:
                eq += q * marks[t]
        return eq


In [ ]:
from numpy.linalg import lstsq
import numpy as np
import pandas as pd

def rolling_beta(y: pd.Series, x: pd.Series, window: int = 60) -> pd.Series:
    betas = []
    idxs = []
    for i in range(window, len(y)+1):
        ys = y.iloc[i-window:i].values.reshape(-1, 1)
        xs = x.iloc[i-window:i].values.reshape(-1, 1)
        # OLS with intercept
        X = np.c_[np.ones_like(xs), xs]
        b, _, _, _ = lstsq(X, ys, rcond=None)
        betas.append(float(b[1]))
        idxs.append(y.index[i-1])
    return pd.Series(betas, index=idxs)

def hedge_ratio_vs_etf(asset: pd.Series, etf: pd.Series, window: int = 60) -> pd.Series:
    asset_ret = asset.pct_change().dropna()
    etf_ret = etf.pct_change().dropna()
    aligned = pd.concat([asset_ret, etf_ret], axis=1, join="inner").dropna()
    aligned.columns = ["asset", "etf"]
    beta = rolling_beta(aligned["asset"], aligned["etf"], window=window)
    return beta.reindex(asset.index).ffill().fillna(0.0)

In [ ]:
import numpy as np
import pandas as pd

def sharpe(returns, rf=0.0):
    r = np.asarray(returns)
    if r.std() == 0:
        return 0.0
    return np.sqrt(252) * (r.mean() - rf/252) / r.std()

def sortino(returns, rf=0.0):
    r = np.asarray(returns)
    downside = r[r < 0]
    denom = downside.std() if downside.size else 0.0
    if denom == 0:
        return 0.0
    return np.sqrt(252) * (r.mean() - rf/252) / denom

def max_drawdown(equity_curve: pd.Series) -> float:
    cummax = equity_curve.cummax()
    dd = (equity_curve - cummax) / cummax
    return float(dd.min()) if len(dd) else 0.0

In [ ]:
# Select tickers
TICKERS = ["AAPL", "MSFT", "GOOGL", "AMZN", "META"]
ETF = "SPY"
INTERVAL = "1d"
PERIOD = "2y"

# Download 'Close' prices only
prices = yf.download(TICKERS + [ETF], interval=INTERVAL, period=PERIOD, progress=False)["Close"]

# Separate assets and ETF
assets = prices[TICKERS]
etf = prices[ETF]

print(assets.head())

In [ ]:
# Extract AAPL close prices for simulation
aapl_prices = prices["AAPL"].dropna()

In [ ]:
# Create mock requests and split train/test sets
reqs = build_price_requests(assets, lot=100)

# Chronological split for training/testing
cut = int(0.7 * len(reqs))
train_reqs = reqs[:cut]
test_reqs = reqs[cut:]

In [ ]:
equity = []
timestamps = []
marks = {}

mm = SimpleMarketMaker(base_spread_bps=10.0, pos_limit=2000, skew_bps_at_limit=50.0)

for (ticker, vol, ref_px, ts) in test_reqs:
    ticker = "AAPL" if random.random() < 0.3 else ticker  # force AAPL trades
    mm.fill_against_stream(ticker, ref_px, vol, ts)
    marks[ticker] = ref_px
    equity.append(mm.mark_to_market(marks))
    timestamps.append(ts)

print("✅ Total completed trades:", len(mm.completed_trades))
if mm.completed_trades:
    print("Example trade:", vars(mm.completed_trades[0]))


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

asset = assets.iloc[:, 0]  # pick first asset for demo
beta = hedge_ratio_vs_etf(asset, etf, window=60)

# Build a hedged return series
asset_ret = asset.pct_change().fillna(0.0)
etf_ret = etf.pct_change().fillna(0.0).reindex(asset_ret.index).fillna(0.0)
hedged_ret = asset_ret - beta.shift(1).fillna(0.0) * etf_ret  # lag beta to avoid peeking

print("Unhedged Sharpe:", round(sharpe(asset_ret), 3))
print("Hedged Sharpe:", round(sharpe(hedged_ret), 3))

plt.figure()
pd.DataFrame({
    "Unhedged": (1+asset_ret).cumprod(),
    "Hedged": (1+hedged_ret).cumprod()
}).plot(title="Cumulative Return: Unhedged vs Hedged")
plt.xlabel("Date")
plt.ylabel("Cumulative Growth")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

pair = ("AAPL", "MSFT")
spread = assets[pair[0]] - assets[pair[1]]
z = (spread - spread.rolling(60).mean()) / (spread.rolling(60).std()+1e-8)

plt.figure()
z.plot(title=f"Z-score Spread: {pair[0]} - {pair[1]}")
plt.axhline(2.0); plt.axhline(-2.0); plt.axhline(0.0)
plt.xlabel("Date"); plt.ylabel("Z-score")
plt.show()

In [ ]:
price_requests = reqs  

test_requests = []

for index in range(0, 10):
    test_requests.append(price_requests[index])

#print(test_requests)

In [ ]:
request_with_prices = []

for price in prices:
    for request in test_requests:
        ticker = request[0]
        date = request[1]
        volume = request[2]

        for price in prices: 
            if price[0] == ticker and price[1] == date: 
                reference_price = price[2]

                request_with_prices.append(([ticker, date, volume], reference_price))
                break

#print(request_with_prices)

In [ ]:
class QuotedTrade:
    def __init__(self, ticker, trade_volume, ref_price, bid_price, offer_price,
                 date):
        self.ticker = ticker
        self.trade_volume = trade_volume
        self.ref_price = ref_price
        self.bid_price = bid_price
        self.offer_price = offer_price
        self.date = date

    def __str__(self):
        return f'Trade Request for {self.ticker}, {self.trade_volume} shares @ {self.ref_price} on {self.date}. Bid Price: {self.bid_price} and Offer Price: {self.offer_price}'

    def __repr__(self):
        return f'QuotedTrade(ticker={self.ticker}, trade_volume={self.trade_volume}, ref_price={self.ref_price}, bid_price={self.bid_price}, offer_price={self.offer_price}, date={self.date})'

In [ ]:
quoted_trades = []

for matched in request_with_prices:
    request=matched[0]
    ref_price=matched[1]

    ticker=request[0]
    date=request[1]
    trade_volume=request[2]

    spread_pct = 0.005
    
    bid_price = ref_price * (1 - spread_pct/2)
    offer_price = ref_price * (1 + spread_pct/2) 
    

    quote = QuotedTrade(ticker=ticker, 
                        trade_volume=trade_volume, 
                        ref_price=ref_price, 
                        bid_price=bid_price, 
                        offer_price=offer_price, 
                        date=date
                        )


    quoted_trades.append(quote)

In [ ]:
hf_responses = []

for trade in quoted_trades:
    response = hf.show(trade) 
    
    hf_responses.append(response)

#print(hf_responses[:5])
#print(hf_responses)

In [ ]:
mm = SimpleMarketMaker(base_spread_bps=10.0, pos_limit=2000, skew_bps_at_limit=50.0)

In [ ]:
import random

equity = []
timestamps = []
marks = {}

# Instantiate your market maker
mm = SimpleMarketMaker(base_spread_bps=10.0, pos_limit=2000, skew_bps_at_limit=50.0)

@dataclass
class SimpleTrade:
    ticker: str
    bid_price: float
    offer_price: float
    ref_price: float
    date: any
    side: str
    volume: float

In [ ]:
# Reset previous trades to avoid duplication
mm.completed_trades = []

# Create aligned bid–ask quotes directly from AAPL close prices
spread_pct = 0.005  # 0.5% spread
for ts, ref_px in zip(aapl_prices.index, aapl_prices.values):
    ticker = "AAPL"
    vol = 100  # arbitrary constant
    bid_price = ref_px * (1 - spread_pct / 2)
    ask_price = ref_px * (1 + spread_pct / 2)

    # Alternate sides randomly
    trade_side = random.choice(["BUY", "SELL"])

    # Save synthetic trade
    mm.completed_trades.append({
        "ticker": ticker,
        "bid_price": bid_price,
        "offer_price": ask_price,
        "ref_price": ref_px,
        "date": ts,
        "side": trade_side,
        "volume": vol
    })

In [ ]:
for quote in quoted_trades:

    mm.add_quoted_trade(quote)

#print(mm.quoted_trades[:2])
#print(mm.quoted_trades)

In [ ]:
class CompletedTrade:

    def __init__ (self, ticker, trade_volume, trade_price, mm_action, ref_price, bid_price, offer_price, date): 
        self.ticker = ticker 
        self.trade_volume = trade_volume 
        self.trade_price = trade_price 
        self.mm_action = mm_action 
        self.ref_price = ref_price 
        self.bid_price = bid_price 
        self.offer_price = offer_price 
        self.date = date 

In [ ]:
for r in hf_responses[:3]:
    print(r.hf_action)
    
for response in hf_responses:
        

    if response.hf_action == "buy": 
        complete_trade = CompletedTrade(
            ticker = response.ticker, 
            trade_volume = response.trade_volume,
            trade_price = response.trade_price,
            mm_action = "buy",
            ref_price = response.ref_price, 
            bid_price = response.bid_price,
            offer_price = response.offer_price,
            date = response.date )

        mm.add_trade(complete_trade)

    elif response.hf_action == "sell": 
        complete_trade = CompletedTrade(
            ticker = response.ticker, 
            trade_volume = response.trade_volume,
            trade_price = response.trade_price,
            mm_action = "sell",
            ref_price = response.ref_price, 
            bid_price = response.bid_price,
            offer_price = response.offer_price,
            date = response.date )
        
        mm.add_trade(complete_trade)

    elif response.hf_action == "refuse": 
        continue

#print(mm.completed_trades[:2])
#print(mm.completed_trades)

In [ ]:
bid_data = []
offer_data = []
quote_dates = []
ref_data = []

for trade in mm.completed_trades:
    # Only consider AAPL stock
    if trade["ticker"] == "AAPL":
        bid_data.append(trade["bid_price"])
        offer_data.append(trade["offer_price"])
        quote_dates.append(trade["date"])
        ref_data.append(trade["ref_price"])

print("bid_data:", bid_data)
print("offer_data:", offer_data)
print("quote_dates:", quote_dates)

In [ ]:
# Show the first few rows of your prices DataFrame
print(prices.head())

# Focus on AAPL data (or another stock if you prefer)
ref_price_data = prices["AAPL"].values.tolist()
ref_dates = prices.index.to_list()

print("✅ Sample reference prices:", ref_price_data[:5])
print("✅ Sample reference dates:", ref_dates[:5])


for price in prices: 
    ref_dates = quote_dates.copy()
    ref_price_data = ref_data.copy()

print("ref_data:", ref_data)
print("ref_dates:", ref_dates)

In [ ]:
import pandas as pd

# 1) Make sure quote_dates are proper timestamps
quote_dates = pd.to_datetime(quote_dates)

# 2) Get the full daily AAPL Close series (already in your 'prices' DataFrame)
aapl_close = prices["AAPL"].sort_index()  # DatetimeIndex -> Close

# 3) Align the AAPL Close to the *trade* timestamps using forward-fill
#    (i.e., for each trade time, use the most recent known close)
ref_aligned = aapl_close.reindex(quote_dates, method="ffill").to_numpy()

# 4) Build a single, aligned DataFrame and sort by time
plot_df = pd.DataFrame({
    "date": quote_dates,
    "Bid Price": pd.to_numeric(bid_data, errors="coerce"),
    "Offer Price": pd.to_numeric(offer_data, errors="coerce"),
    "Reference Price": ref_aligned
}).dropna().sort_values("date")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(plot_df["date"], plot_df["Reference Price"], label="Reference Price (AAPL Close)")
plt.plot(plot_df["date"], plot_df["Bid Price"], label="Bid (Simulated)")
plt.plot(plot_df["date"], plot_df["Offer Price"], label="Ask (Simulated)")

plt.title("AAPL Bid, Offer, and Reference Prices Over Time")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()  
plt.show()

In [ ]:
class QuotedTrade:
    def __init__(self, ticker, trade_volume, ref_price, bid_price, offer_price, date):
        self.ticker = ticker
        self.trade_volume = trade_volume
        self.ref_price = ref_price
        self.bid_price = bid_price
        self.offer_price = offer_price
        self.date = date
        
class CompletedTrade:
    def __init__(self, ticker, trade_volume, trade_price, mm_action, ref_price, bid_price, offer_price, date):
        self.ticker = ticker
        self.trade_volume = trade_volume
        self.trade_price = trade_price
        self.mm_action = mm_action
        self.ref_price = ref_price
        self.bid_price = bid_price
        self.offer_price = offer_price
        self.date = date 

In [ ]:
import random

class MockHFTrader:
    """Simple high-frequency trader that randomly accepts or rejects quotes."""
    def show(self, quote):
        # Randomly decide whether to accept or refuse the quote
        action = random.choice(["Buy", "Sell", "Refused"])
        
        # Return a simple object with expected attributes
        return type("HFResponse", (), {
            "hf_action": action,
            "trade_price": random.uniform(quote.bid_price, quote.offer_price)
        })()

# ✅ Instantiate the mock trader
hf = MockHFTrader()


In [ ]:
for i, request in enumerate(price_requests): 
    ticker = request[0]
    trade_volume = request[1]
    ref_price = request[2]
    date = i

    # ✅ Initialize position if ticker not in current_positions
    if ticker not in mm.current_positions:
        # Create a simple object with a position_volume attribute
        mm.current_positions[ticker] = type("Pos", (), {"position_volume": 0})()

    position = mm.current_positions[ticker].position_volume

    spread = 2 
    skew = 0.5

    bid_price = ref_price - spread
    offer_price = ref_price + spread

    if position > 0:  # Long
        bid_price -= skew
        offer_price -= skew 
    elif position < 0:  # Short 
        bid_price += skew 
        offer_price += skew 

    quote = QuotedTrade(
        ticker=ticker, 
        trade_volume=trade_volume, 
        ref_price=ref_price, 
        bid_price=bid_price, 
        offer_price=offer_price, 
        date=date
    )

    mm.add_quoted_trade(quote)

    response = hf.show(quote)

    if response.hf_action != "Refused":
        completed_trade = CompletedTrade(
            ticker=ticker,
            trade_volume=trade_volume,
            trade_price=response.trade_price,
            mm_action=response.hf_action,
            ref_price=ref_price,
            bid_price=bid_price,
            offer_price=offer_price,
            date=date
        )

        mm.add_trade(completed_trade)

            # ✅ Optional improvement: update position dynamically
        if response.hf_action == "Buy":
            mm.current_positions[ticker].position_volume += trade_volume
        elif response.hf_action == "Sell":
            mm.current_positions[ticker].position_volume -= trade_volume

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# ---- SETTINGS ----
chosen_ticker = "FB"  # Change if needed

# ---- EXTRACT DATA ----
quote_dates = []
bid_prices = []
offer_prices = []
ref_prices = []

# Collect all quotes for the chosen ticker
for quote in getattr(mm, "quoted_trades", []):
    if quote.ticker == chosen_ticker:
        quote_dates.append(quote.date)
        bid_prices.append(quote.bid_price)
        offer_prices.append(quote.offer_price)
        ref_prices.append(quote.ref_price)

# ---- FALLBACK: AUTO-DETECT ACTIVE TICKER ----
if not quote_dates:
    all_tickers = [q.ticker for q in getattr(mm, "quoted_trades", [])]
    if all_tickers:
        most_common = pd.Series(all_tickers).mode()[0]
        print(f"No quotes found for '{chosen_ticker}'. Using '{most_common}' instead.")
        chosen_ticker = most_common
        quote_dates, bid_prices, offer_prices, ref_prices = [], [], [], []
        for quote in mm.quoted_trades:
            if quote.ticker == chosen_ticker:
                quote_dates.append(quote.date)
                bid_prices.append(quote.bid_price)
                offer_prices.append(quote.offer_price)
                ref_prices.append(quote.ref_price)
    else:
        print("⚠️ No quotes found at all — make sure your simulation produced any trades.")
        raise SystemExit

# ---- CONVERT TO DATAFRAME FOR SAFETY ----
plot_df = pd.DataFrame({
    "Date": quote_dates,
    "Bid": bid_prices,
    "Offer": offer_prices,
    "Reference": ref_prices
}).dropna().sort_values("Date")

# ---- VALIDATION ----
if plot_df.empty:
    print(f"⚠️ No valid data available for {chosen_ticker}.")
else:
    # ---- PLOT ----
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(plot_df["Date"], plot_df["Bid"], label="Bid Price", color="tab:blue")
    ax.plot(plot_df["Date"], plot_df["Offer"], label="Offer Price", color="tab:orange")
    ax.plot(plot_df["Date"], plot_df["Reference"], label="Reference Price", color="tab:green")

    ax.set_title(f"{chosen_ticker} Bid, Offer and Reference Prices Over Time")
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
import pandas as pd

print("🔍 Checking structure of 'prices' variable...")
print("-" * 60)

# Check type
print("Type of 'prices':", type(prices))

# Case 1: if 'prices' is a pandas DataFrame
if isinstance(prices, pd.DataFrame):
    print(f"✅ Detected DataFrame with shape: {prices.shape}")
    print("Columns:", list(prices.columns))
    print("\nFirst 5 rows:")
    print(prices.head())

    # Example check — if it has at least 3 columns
    if prices.shape[1] >= 3:
        example = prices.iloc[0].tolist()
        print("\nExample row:", example)
    else:
        print("⚠️ DataFrame does not have at least 3 columns (expected: Ticker, Date, RefPrice).")

# Case 2: if 'prices' is a list
elif isinstance(prices, list):
    print(f"✅ Detected list with {len(prices)} elements.")
    if len(prices) > 0:
        print("Example entry:", prices[0])
        if isinstance(prices[0], (list, tuple)):
            print(f"Entry length: {len(prices[0])} → {prices[0]}")
        else:
            print("⚠️ Entries are not lists/tuples. Expected e.g. ['AAPL', date, price].")
    else:
        print("⚠️ 'prices' list is empty!")

# Case 3: Unexpected type
else:
    print("⚠️ 'prices' is neither a list nor a DataFrame — unexpected type.")


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# --- Compute daily % change for all tickers ---
returns = prices.pct_change().dropna()

print("✅ Computed daily percentage changes:")
print(returns.head())

# --- Example: select AAPL (or any other ticker) ---
chosen_ticker = "AAPL"

if chosen_ticker in returns.columns:
    plt.figure(figsize=(10, 5))
    plt.plot(returns.index, returns[chosen_ticker], label=f"{chosen_ticker} Daily % Change", color="tab:blue")
    plt.title(f"{chosen_ticker} Daily Percentage Change Over Time")
    plt.xlabel("Date")
    plt.ylabel("Daily Return")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print(f"⚠️ Ticker '{chosen_ticker}' not found in DataFrame columns.")


In [ ]:
AAPL_returns = AAPL_change.squeeze()
FB_returns = FB_change.squeeze()
GOOGL_returns = GOOGL_change.squeeze()
AMZN_returns = AMZN_change.squeeze()
NFLX_returns = NFLX_change.squeeze()
FAANG_returns = FAANG_change.squeeze()

var_faang = FAANG_returns.var()

cov_aapl = AAPL_returns.cov(FAANG_returns) 
cov_fb = FB_returns.cov(FAANG_returns)
cov_googl = GOOGL_returns.cov(FAANG_returns)
cov_amzn = AMZN_returns.cov(FAANG_returns)
cov_nflx = NFLX_returns.cov(FAANG_returns)

beta_aapl = cov_aapl / var_faang
beta_fb = cov_fb / var_faang
beta_googl = cov_googl / var_faang
beta_amzn = cov_amzn / var_faang
beta_nflx = cov_nflx / var_faang

#print(beta_aapl, beta_fb, beta_googl, beta_amzn, beta_nflx)

In [ ]:
class ExchangeTrade:
    def __init__(self, ticker, trade_volume, ref_price, action, date):
        self.ticker = ticker
        self.trade_volume = trade_volume
        self.ref_price = ref_price
        self.action = action
        self.date = date

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

all_trades = []

# ✅ Handle completed trades
for trade in mm.completed_trades:
    # Use dict-style if it's a dictionary, otherwise attribute-style
    if isinstance(trade, dict):
        date = trade.get("date", None)
        ticker = trade.get("ticker", None)
        volume = trade.get("trade_volume", trade.get("position_volume", 0))
        price = trade.get("trade_price", trade.get("ref_price", trade.get("open_price", 0)))
        action = trade.get("action", trade.get("mm_action", "buy"))
    else:
        date = getattr(trade, "date", None)
        ticker = getattr(trade, "ticker", None)
        volume = getattr(trade, "trade_volume", getattr(trade, "position_volume", 0))
        price = getattr(trade, "trade_price", getattr(trade, "ref_price", getattr(trade, "open_price", 0)))
        action = getattr(trade, "action", getattr(trade, "mm_action", "buy"))
    
    all_trades.append({
        "date": date,
        "ticker": ticker,
        "volume": volume,
        "price": price,
        "action": action
    })

# ✅ Handle hedge trades (same logic)
for trade in getattr(mm, "hedge_trades", []):
    if isinstance(trade, dict):
        date = trade.get("date", None)
        ticker = trade.get("ticker", None)
        volume = trade.get("trade_volume", trade.get("position_volume", 0))
        price = trade.get("trade_price", trade.get("ref_price", trade.get("open_price", 0)))
        action = trade.get("action", trade.get("mm_action", "buy"))
    else:
        date = getattr(trade, "date", None)
        ticker = getattr(trade, "ticker", None)
        volume = getattr(trade, "trade_volume", getattr(trade, "position_volume", 0))
        price = getattr(trade, "trade_price", getattr(trade, "ref_price", getattr(trade, "open_price", 0)))
        action = getattr(trade, "action", getattr(trade, "mm_action", "buy"))
    
    all_trades.append({
        "date": date,
        "ticker": ticker,
        "volume": volume,
        "price": price,
        "action": action
    })

# --- Convert to DataFrame ---
df = pd.DataFrame(all_trades)

if df.empty:
    print("⚠️ No trades found in completed_trades or hedge_trades.")
else:
    # --- Ensure date column is consistent ---
    # Convert to datetime if possible; otherwise keep numeric fallback
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")  # convert what can be dates
        if df["date"].isna().all():  # all NaT → fallback to numeric order
            df["date"] = range(len(df))
    else:
        df["date"] = range(len(df))

    # --- Compute signed trade value ---
    df["signed_value"] = df.apply(
        lambda x: x["volume"] * x["price"] * (1 if str(x["action"]).lower() == "buy" else -1),
        axis=1
    )

    # --- Sort safely by date ---
    df_sorted = df.sort_values(by="date", key=lambda s: pd.to_numeric(s, errors="coerce"))

    # --- Cumulative nominal risk ---
    df_sorted["cumulative_risk"] = df_sorted["signed_value"].cumsum()

    # --- Plot cumulative nominal risk over time ---
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(df_sorted["date"], df_sorted["cumulative_risk"], label="Total Nominal Risk ($)")
    ax.axhline(0, color="black", linestyle="--", linewidth=1)
ax.set_title("Total Net Position (Nominal Risk) over Time")
ax.set_xlabel("Date")
ax.set_ylabel("Combined Nominal Risk Value ($)")
ax.legend()
plt.show()


In [ ]:
print("Completed trades:", len(mm.completed_trades))
print("Hedge trades:", len(mm.hedge_trades))

In [ ]:
prices_FB = []
prices_AAPL = []
prices_AMZN = []
prices_NFLX =[]
prices_GOOGL = []
prices_FAANG = []

for price in prices:
    if price[0] == "AAPL":
        prices_AAPL.append(price)
    elif price[0] == "FB":
        prices_FB.append(price)
    elif price[0] == "GOOGL":
        prices_GOOGL.append(price)
    elif price[0] == "AMZN":
        prices_AMZN.append(price)
    elif price[0] == "NFLX":
        prices_NFLX.append(price)
    elif price[0] == "FAANG":
        prices_FAANG.append(price)

def extract_price_column(price_list):
    # Expect each element like: [ticker, date, price]
    df = pd.DataFrame(price_list)
    if df.shape[1] >= 3:
        return df.iloc[:, 2]   # third column safely
    else:
        print("⚠️ Not enough columns in DataFrame:", df.shape)
        return pd.Series(dtype=float)

FB_df = extract_price_column(prices_FB)
AAPL_df = extract_price_column(prices_AAPL)
AMZN_df = extract_price_column(prices_AMZN)
GOOGL_df = extract_price_column(prices_GOOGL)
NFLX_df = extract_price_column(prices_NFLX)
FAANG_df = extract_price_column(prices_FAANG)

In [ ]:
synthetic_etf = (FB_df + AAPL_df + AMZN_df + NFLX_df + GOOGL_df) / 5
spread = FAANG_df - synthetic_etf

spread_mean = spread.mean()
spread_std = spread.std()
zscore = (spread - spread_mean) / spread_std

threshold = 2
arbitrage_points = zscore[(zscore > threshold) | (zscore < -threshold)]

In [ ]:
fig, axes = plt.subplots(2, 1, figsize= (10, 6), sharex=True) 

axes[0].plot(FAANG_df.index, FAANG_df, label="Real FAANG ETF (Actual)") 
axes[0].plot(synthetic_etf.index, synthetic_etf, label="Synthetic ETF (Fair Value)") 
axes[0].set_title("Real vs. Synthetic FAANG ETF Prices") 
axes[0].set_ylabel("Price")
axes[0].legend()


axes[1].plot(spread.index, spread, label="Spread") 
axes[1].axhline(spread_mean, color="black", linestyle="--", linewidth=1) 
axes[1].axhline(spread_mean + threshold * spread_std, color ="blue", linestyle="--", linewidth=1)
axes[1].axhline(spread_mean - threshold * spread_std, color ="blue", linestyle="--", linewidth=1)
axes[1].set_title("Spread between Real and Synthetic ETF") 
axes[1].set_xlabel("Date") 
axes[1].set_ylabel("Spread Value") 
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np 

FAANGdf = prices["FAANG"]
FBdf    = prices["FB"]
AAPLdf  = prices["AAPL"]
AMZNdf  = prices["AMZN"]
NFLXdf  = prices["NFLX"]
GOOGLdf = prices["GOOGL"]

syntheticetf = (FBdf + AAPLdf + AMZNdf + NFLXdf + GOOGLdf) / 5

for date in FAANGdf.index: 
    real_price = FAANGdf.loc[date]
    synthetic_price = syntheticetf.loc[date]

    threshold = 0.005 * synthetic_price 

    if real_price < synthetic_price - threshold: 
        hedge_fund.execute_order("FAANG", 5, "BUY", date) 

        hedge_fund.execute_order("FB", 1, "SELL", date)
        hedge_fund.execute_order("AAPL", 1, "SELL", date)
        hedge_fund.execute_order("AMZN", 1, "SELL", date)
        hedge_fund.execute_order("NFLX", 1, "SELL", date)
        hedge_fund.execute_order("GOOGL", 1, "SELL", date)

    elif real_price > synthetic_price + threshold: 
        hedge_fund.execute_order("FAANG", 5, "SELL", date) 

        hedge_fund.execute_order("FB", 1, "BUY", date)
        hedge_fund.execute_order("AAPL", 1, "BUY", date)
        hedge_fund.execute_order("AMZN", 1, "BUY", date)
        hedge_fund.execute_order("NFLX", 1, "BUY", date)
        hedge_fund.execute_order("GOOGL", 1, "BUY", date)

In [ ]:
pnl = 0
for ticker in prices:
    pnl += hedge_fund.current_positions[ticker].profit_loss - hedge_fund.current_positions[ticker].commission_costs

print("Ending Balance: ", hedge_fund.balance)

print("Total Profit or Loss (including commission):", pnl)